In [21]:
%%time
import pandas as pd
import requests
import lxml.html as LH
import re                               # For regex
import datetime                         # For days to expiry calculation

import json                             # For volatility
from bs4 import BeautifulSoup

from math import sqrt, exp, log, erf    # For Black Scholes

from decimal import *
getcontext().prec = 5


###########################################
####   Assemble Black and Scholes variables
###########################################

# Expiry Dates List
exp_dates = pd.Series(['30NOV2017', '28DEC2017', '25JAN2018'], name="Expiry")

# Create Symbol Lot Margin (symlotmargin)
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl)[1]          # It's the second table in the url

# Make first row as header
new_header = paisa.iloc[0]
paisa = paisa[1:]
paisa.columns = new_header

symlotmargin = paisa.loc[:, ["Symbol", "Mlot", "TotMgnPerLt"]]

# Merge dates to symlotmargin
# symlotmargin.merge(exp_dates.to_frame(),left_index=index,right_index=index)

# Interest rate (10-year bond yield)
inturl = requests.get("https://countryeconomy.com/bonds/india").content
intrate = float(LH.fromstring(inturl).find_class('numero')[0].text)/100

# Script name
scrip = 'PFC'
option = 'OPTSTK'
exp_date = '28DEC2017'

# NSE URL
url = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument="


# URL for options table        
url = url + option + "&symbol=" + scrip + "&date="+ exp_date 

# Options table
html = requests.get(url).content
nsetable = pd.read_html(html, attrs = {'id':'octable'}, header=1)[-1][:-1].drop(['Chart', 'Chart.1'], 1)

# Current underlying price
tree = LH.fromstring(html)
node = tree.xpath("//b")[0]
undprice = float(re.findall('\d+\.\d+', node.text)[0])
undprice

# Days to expiry
expiry = (datetime.datetime.strptime(exp_date, "%d%b%Y").date() - datetime.datetime.now().date()).days

# Volatility
future = 'FUTSTK'

vol_url = "https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?instrument=" + \
         future + "&underlying=" + scrip

url = "https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?underlying=PFC&instrument=FUTSTK"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
data=soup.find(id="responseDiv").text.strip()
d1 = json.loads(data)
sigma=float(d1['data'][0]['annualisedVolatility'])/100


# Dividend rate
divurl = "https://finance.google.com/finance?q=NSE:"+scrip
page = requests.get(divurl)
root = LH.fromstring(page.content)
dividend = float(root.findall('.//table')[2].text_content().strip().split("\n")[2].split('/')[0])/100

df = {"undprice": undprice, "strike": 100.0, "time": expiry, "rate": intrate, "sigma": sigma, "divrate": dividend}

def calc_bs(undprice, strike, time, rate, sigma, divrate):

    #statistics
    sigTsquared = sqrt(Decimal(time)/365)*sigma
    edivT = exp((-divrate*time)/365)
    ert = exp((-rate*time)/365)
    d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #outputs
    callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
    
    return callPrice, putPrice

Wall time: 7.84 s


In [4]:
symlotmargin[symlotmargin['Symbol'] == 'PFC']

,Symbol,Mlot,TotMgnPerLt
70,PFC,6000,91095


In [22]:
calc_bs(**df)

(16.4, 0.04)

6.0

In [8]:
undprice

116.5

In [10]:
expiry

16

In [11]:
intrate

0.0722

In [15]:
dividend

6.0